In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeDriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from typing import Union

from tqdm import tqdm
import pandas as pd

def wait_and_find_element(element : Union[WebDriver, WebElement], xpath):
    WebDriverWait(element, 4).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return element.find_element(By.XPATH, xpath)

def wait_and_find_elements(element : Union[WebDriver, WebElement], xpath):
    WebDriverWait(element, 4).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return element.find_elements(By.XPATH, xpath)

options = ChromeOptions()
# options.add_argument('--headless')
options.add_argument("--window-size=2560,1440")
# options.add_argument('--no-sandbox')
url="https://sjpt.sejong.ac.kr/websquare/websquare.html?w2xPath=/sch/ext/SueOpenTimeQ.xml&&p_id=mf_"
service = ChromeService(ChromeDriverManager().install())
driver = ChromeDriver(service=service, options=options)
driver.get(url)


row_size = 20 #zoom 80% 기준


#리스트 갱신
wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()

#학과 선택
major_list = driver.find_elements(By.XPATH, '//tbody//*[@class="w2input_table_sb_row"]')

wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()

class_list = []

for major_idx in tqdm(range(2, len(major_list)+1)):
    wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()
    wait_and_find_element(driver, f'//*[@class="w2input_table_sb w2input_table"]//tbody//tr[{major_idx}]').click()
    wait_and_find_element(driver, '//*[@class="w2group w2tb_td btn_grp w2tb_noTH"]//*[@class="w2anchor2 btn_sch"]').click()

    time.sleep(0.5)
    #스크롤이 없을 경우 wait_and_find_element함수에 안걸려서 find_element(s) 사용
    
    scroll_op = driver.find_element(By.XPATH, '//*[@class="w2grid_scrollY"]')
    scroll_height = int(driver.find_element(By.XPATH, '//*[@class="w2grid_scrollY"]//div').size['height'])
    row_cnt = (scroll_height-268)/20

    time.sleep(0.5)
    if scroll_height == 0:
        # table_data_contents = wait_and_find_elements(driver, '//*[@id="mf_grd_main_body_tbody"]//tr')
        table_data_contents = driver.find_elements(By.XPATH, '//*[@id="mf_grd_main_body_tbody"]//tr')
        for row in table_data_contents:
            # row_data = wait_and_find_elements(row, './/td')
            row_data = row.find_elements(By.XPATH, './/td')
            tmp_list = []
            for shell in row_data:
                tmp_list.append(shell.text)
            class_list.append(tmp_list)
            
    else:
        for table_data in range(scroll_height // (row_size*20)):
            table_data_contents = wait_and_find_elements(driver, '//*[@id="mf_grd_main_body_tbody"]//tr')
            # table_data_contents = driver.find_elements(By.XPATH, '//*[@id="mf_grd_main_body_tbody"]//tr')
            for row in table_data_contents:
                # row_data = wait_and_find_elements(row, './/td')
                row_data = row.find_elements(By.XPATH, './/td')
                tmp_list = []
                for shell in row_data:
                    tmp_list.append(shell.text)
                class_list.append(tmp_list)
            
            driver.execute_script(f"arguments[0].scrollBy(0, {row_size*20})", scroll_op) #mf_grd_main_size_y에서 스크롤 크기 확인 가능

pd.DataFrame(class_list).to_csv('table.csv', encoding='cp949')

# for table_data in range(scroll_height // (row_size*20)):
#     time.sleep(1)
#     for row in table_data_contents:
#         row_data = row.find_elements(By.XPATH, './/td')
#         tmp_list = []
#         for shell in row_data:
#             tmp_list.append(shell.text)
#         class_list.append(tmp_list)
    
#     driver.execute_script(f"arguments[0].scrollBy(0, {row_size*20})", itemlist) #mf_grd_main_size_y에서 스크롤 크기 확인 가능

KeyboardInterrupt: 

In [5]:
df = pd.read_csv('./table.csv', encoding ='utf-8')
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,0,1.0,AI로봇학과,011839,1.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,양해상,화19:30-20:30,센B107,NaN,NaN,NaN,NaN
1,1,2.0,AI로봇학과,011839,2.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,조재진,화19:30-20:30,센B205,NaN,NaN,NaN,NaN
2,2,3.0,AI로봇학과,011839,3.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,양해상,화20:30-21:30,센B107,NaN,NaN,NaN,NaN
3,3,4.0,AI로봇학과,000304,1.0,공업수학1,NaN,전기,NaN,3.0 / 3 / 0,...,교과목개요,수업계획서,AI로봇학과,이찬화,화목15:00-16:30,센B106,NaN,NaN,NaN,NaN
4,4,5.0,AI로봇학과,000304,2.0,공업수학1,영어/한국어,전기,NaN,3.0 / 3 / 0,...,교과목개요,수업계획서,AI로봇학과,박규민,화목16:30-18:00,센B204,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,3313,31.0,회화과,004876,1.0,한.종합실기2,NaN,전필,NaN,3.0 / 2 / 2,...,교과목개요,수업계획서,회화과,안소연,월10:30-14:30,진201,NaN,NaN,NaN,NaN
3314,3314,32.0,회화과,005764,1.0,졸업작품(P/NP),NaN,전필,NaN,0.0 / 0 / 0,...,교과목개요,수업계획서,회화과,이강화,NaN,진301,NaN,NaN,NaN,NaN
3315,3315,33.0,회화과,001636,1.0,서.종합실기4,NaN,전선,NaN,2.0 / 2 / 2,...,교과목개요,수업계획서,회화과,송은영,월15:00-19:00,진202,NaN,NaN,NaN,NaN
3316,3316,34.0,회화과,004877,1.0,한.종합실기4,NaN,전선,NaN,2.0 / 2 / 2,...,교과목개요,수업계획서,회화과,정재호,수12:00-16:00,진201,NaN,NaN,NaN,NaN


In [9]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo")
prompt = PromptTemplate(
    input_variables=["course_name"],
    template="수업 이름을 보고 수업에 대한 설명을 2줄 정도로 생성해줘: {course_name}."
)

chain = LLMChain(llm=llm, prompt=prompt)
def generate_description(course_name):
    description = chain.run(course_name)
    return description
generate_description('현대사회이론')

C:\Users\user\AppData\Local\Temp\ipykernel_20152\1794050600.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo")


'현대사회이론은 현대 사회 현상과 구조를 이해하고 분석하는 이론적 접근 방식을 다루는 수업입니다. 주요 사회 이론가들의 이론을 토대로 현대 사회의 다양한 측면을 탐구하며 사회적 현상을 해석하는 능력을 강화합니다.'

In [1]:
#preprocessing
import pandas as pd
import numpy as np
table_df = pd.read_csv('./source/data/table.csv', encoding ='utf-8')


In [2]:
col_name = ['개설학과전공', '학수번호', '분반', '교과목명',
            '강의언어', '이수구분', '선택영역', '학점/이론/실습',
            '학년(학기)', '대상과정', '교과목개요', '수업계획서',
            '주관학과', '교수명', '요일 및 강의시간', '강의실', '사이버강좌']
tmp_df = table_df.iloc[:,2:-3].loc[~table_df.iloc[:,2:-4]['1'].isnull()]
tmp_df.columns = col_name
tmp_df['학점'] = tmp_df['학점/이론/실습'].apply(lambda x : round(float(x.split('/')[0].strip())))
for col in ['분반', '학년(학기)', '학점']: tmp_df[col] = tmp_df[col].astype(np.int8)
tmp_df.fillna('', inplace=True)
tmp_df.reset_index(drop=True, inplace=True)
tmp_df.to_csv('./source/preprocessing_table.csv')


In [14]:
course_name_list = list(tmp_df['교과목명'].unique())
course_description_list = []
for course_name in course_name_list:
    course_description_list.append(generate_description(course_name))

In [22]:
gpt_df = pd.DataFrame({'교과목명':course_name_list, '교과목설명':course_description_list})
gpt_df.to_csv('./source/generate_course_description.csv')

In [23]:
pd.merge(tmp_df, gpt_df, on='교과목명', how='inner').to_csv('./source/generate_preprocessing_table.csv')

In [ ]:
gpt_df

In [15]:
course_description_list

['세종인들을 위해 다양한 전공 영역을 탐색하고 선택하는 과정을 도와주는 수업입니다. 학생들이 자신에게 맞는 전공을 찾아가는데 도움을 줄 것입니다.',
 '공업수학1은 공학 분야에서 필요한 수학적 개념과 이론을 다루는 과목으로, 미적분학과 선형대수학 등을 중심으로 학습합니다. 엔지니어링 계산과 문제 해결을 위한 수학적 기초를 다지는데 중점이 있습니다.',
 '선형대수는 벡터, 행렬, 선형변환 등을 연구하는 수학의 한 분야로, 다양한 분야에서 활용되는 중요한 개념들을 다룹니다. 선형대수는 현대 수학의 기초가 되는 중요한 이론으로, 컴퓨터 공학, 물리학, 통계학 등 다양한 분야에 응용됩니다.',
 '고급C프로그래밍및실습은 C언어를 활용한 고급 프로그래밍 기술을 학습하고, 실제 프로젝트를 통해 응용력을 향상시키는 수업입니다. C언어의 고급 기능과 프로그래밍 기법을 익히며 다양한 실습을 통해 실무에 적용할 수 있는 능력을 키우는 과정입니다.',
 'SM-디지털영상의 합성과 편집 수업은 영상 제작에 필요한 디지털 기술과 편집 기법을 배우며, 다양한 소프트웨어를 통해 영상을 합성하고 편집하는 방법을 습득할 수 있는 수업입니다. 실무적인 영상 제작 능력을 향상시키고, 창의적인 아이디어를 표현하는데 도움이 됩니다.',
 '이 수업은 학생들이 전공 지식을 활용하여 현업에서 발생하는 문제를 해결하는 디자인 프로젝트를 수행하는 과목입니다. 산업체 현장에서의 경험을 통해 실무 능력을 향상시키고 졸업 프로젝트를 완성하는 기회를 제공합니다.',
 'ST-캡스톤디자인은 학생들이 전공지식을 활용하여 현업에서 발생하는 문제를 해결하고 창의적인 디자인을 통해 프로젝트를 완성하는 수업입니다. 산업체 현장에서의 경험을 통해 실무 능력을 키우고 팀원들과 협력하며 문제를 해결하는 역량을 키우는 수업입니다.',
 '시스템생명공학캡스톤디자인Ⅱ 수업은 학생들에게 현장에서 발생하는 실제 문제를 해결하기 위한 시스템생명공학 기술을 적용하는 경험을 제공합니다. 학생들은 팀을 이루어 프로젝트를 기획하고 실행하며

In [6]:
generate_description('현대사회이론')

BadRequestError: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [4]:
tmp_df

,개설학과전공,학수번호,분반,교과목명,강의언어,이수구분,선택영역,학점/이론/실습,학년(학기),대상과정,교과목개요,수업계획서,주관학과,교수명,요일 및 강의시간,강의실,사이버강좌,학점
0,AI로봇학과,011839,1,세종인을위한전공탐색,,공필,,1.0 / 0 / 1,1,학사,교과목개요,수업계획서,AI로봇학과,양해상,화19:30-20:30,센B107,,1
1,AI로봇학과,011839,2,세종인을위한전공탐색,,공필,,1.0 / 0 / 1,1,학사,교과목개요,수업계획서,AI로봇학과,조재진,화19:30-20:30,센B205,,1
2,AI로봇학과,011839,3,세종인을위한전공탐색,,공필,,1.0 / 0 / 1,1,학사,교과목개요,수업계획서,AI로봇학과,양해상,화20:30-21:30,센B107,,1
3,AI로봇학과,000304,1,공업수학1,,전기,,3.0 / 3 / 0,1,학사,교과목개요,수업계획서,AI로봇학과,이찬화,화목15:00-16:30,센B106,,3
4,AI로봇학과,000304,2,공업수학1,영어/한국어,전기,,3.0 / 3 / 0,1,학사,교과목개요,수업계획서,AI로봇학과,박규민,화목16:30-18:00,센B204,,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,회화과,001631,2,서.종합실기2,,전필,,3.0 / 2 / 2,4,학사,교과목개요,수업계획서,회화과,이강화,수15:00-19:00,진202,,3
2524,회화과,004876,1,한.종합실기2,,전필,,3.0 / 2 / 2,4,학사,교과목개요,수업계획서,회화과,안소연,월10:30-14:30,진201,,3
2525,회화과,005764,1,졸업작품(P/NP),,전필,,0.0 / 0 / 0,4,학사,교과목개요,수업계획서,회화과,이강화,,진301,,0
2526,회화과,001636,1,서.종합실기4,,전선,,2.0 / 2 / 2,4,학사,교과목개요,수업계획서,회화과,송은영,월15:00-19:00,진202,,2


In [10]:

table_df.drop(columns=['Unnamed: 0','0']).rename()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,AI로봇학과,011839,1.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,1.0,학사,교과목개요,수업계획서,AI로봇학과,양해상,화19:30-20:30,센B107,NaN,NaN,NaN,NaN
1,AI로봇학과,011839,2.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,1.0,학사,교과목개요,수업계획서,AI로봇학과,조재진,화19:30-20:30,센B205,NaN,NaN,NaN,NaN
2,AI로봇학과,011839,3.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,1.0,학사,교과목개요,수업계획서,AI로봇학과,양해상,화20:30-21:30,센B107,NaN,NaN,NaN,NaN
3,AI로봇학과,000304,1.0,공업수학1,NaN,전기,NaN,3.0 / 3 / 0,1.0,학사,교과목개요,수업계획서,AI로봇학과,이찬화,화목15:00-16:30,센B106,NaN,NaN,NaN,NaN
4,AI로봇학과,000304,2.0,공업수학1,영어/한국어,전기,NaN,3.0 / 3 / 0,1.0,학사,교과목개요,수업계획서,AI로봇학과,박규민,화목16:30-18:00,센B204,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,회화과,004876,1.0,한.종합실기2,NaN,전필,NaN,3.0 / 2 / 2,4.0,학사,교과목개요,수업계획서,회화과,안소연,월10:30-14:30,진201,NaN,NaN,NaN,NaN
3314,회화과,005764,1.0,졸업작품(P/NP),NaN,전필,NaN,0.0 / 0 / 0,4.0,학사,교과목개요,수업계획서,회화과,이강화,NaN,진301,NaN,NaN,NaN,NaN
3315,회화과,001636,1.0,서.종합실기4,NaN,전선,NaN,2.0 / 2 / 2,4.0,학사,교과목개요,수업계획서,회화과,송은영,월15:00-19:00,진202,NaN,NaN,NaN,NaN
3316,회화과,004877,1.0,한.종합실기4,NaN,전선,NaN,2.0 / 2 / 2,4.0,학사,교과목개요,수업계획서,회화과,정재호,수12:00-16:00,진201,NaN,NaN,NaN,NaN


In [7]:
df.loc[~df['1'].isnull('Unnamed: 0,0')]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,0,1.0,AI로봇학과,011839,1.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,양해상,화19:30-20:30,센B107,NaN,NaN,NaN,NaN
1,1,2.0,AI로봇학과,011839,2.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,조재진,화19:30-20:30,센B205,NaN,NaN,NaN,NaN
2,2,3.0,AI로봇학과,011839,3.0,세종인을위한전공탐색,NaN,공필,NaN,1.0 / 0 / 1,...,교과목개요,수업계획서,AI로봇학과,양해상,화20:30-21:30,센B107,NaN,NaN,NaN,NaN
3,3,4.0,AI로봇학과,000304,1.0,공업수학1,NaN,전기,NaN,3.0 / 3 / 0,...,교과목개요,수업계획서,AI로봇학과,이찬화,화목15:00-16:30,센B106,NaN,NaN,NaN,NaN
4,4,5.0,AI로봇학과,000304,2.0,공업수학1,영어/한국어,전기,NaN,3.0 / 3 / 0,...,교과목개요,수업계획서,AI로봇학과,박규민,화목16:30-18:00,센B204,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,3312,30.0,회화과,001631,2.0,서.종합실기2,NaN,전필,NaN,3.0 / 2 / 2,...,교과목개요,수업계획서,회화과,이강화,수15:00-19:00,진202,NaN,NaN,NaN,NaN
3313,3313,31.0,회화과,004876,1.0,한.종합실기2,NaN,전필,NaN,3.0 / 2 / 2,...,교과목개요,수업계획서,회화과,안소연,월10:30-14:30,진201,NaN,NaN,NaN,NaN
3314,3314,32.0,회화과,005764,1.0,졸업작품(P/NP),NaN,전필,NaN,0.0 / 0 / 0,...,교과목개요,수업계획서,회화과,이강화,NaN,진301,NaN,NaN,NaN,NaN
3315,3315,33.0,회화과,001636,1.0,서.종합실기4,NaN,전선,NaN,2.0 / 2 / 2,...,교과목개요,수업계획서,회화과,송은영,월15:00-19:00,진202,NaN,NaN,NaN,NaN


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeDriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from typing import Union

from tqdm import tqdm
import pandas as pd
import os

def wait_and_find_element(element : Union[WebDriver, WebElement], xpath):
    WebDriverWait(element, 4).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return element.find_element(By.XPATH, xpath)

def wait_and_find_elements(element : Union[WebDriver, WebElement], xpath):
    WebDriverWait(element, 4).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    return element.find_elements(By.XPATH, xpath)

options = ChromeOptions()
# options.add_argument('--headless')
options.add_argument("--window-size=2560,1440")
# options.add_argument('--no-sandbox')
url="https://sjpt.sejong.ac.kr/websquare/websquare.html?w2xPath=/sch/ext/SueOpenTimeQ.xml&&p_id=mf_"
service = ChromeService(ChromeDriverManager().install())
prefs = {"download.default_directory" : r"C:\Users\user\Documents\lets_girin\source\course"}
options.add_experimental_option("prefs",prefs)
driver = ChromeDriver(service=service, options=options)
driver.get(url)


row_size = 20 #zoom 80% 기준


#리스트 갱신
wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()

#학과 선택
major_list = driver.find_elements(By.XPATH, '//tbody//*[@class="w2input_table_sb_row"]')

wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()

class_list = []
cnt = 21
for major_idx in tqdm(range(6, len(major_list)+1)):
    wait_and_find_element(driver, '//tr[@class="w2group  w2tb_tr"][2]//*[@class="w2group w2tb_td"]//*[@class="w2autoComplete_col_button"]').click()
    wait_and_find_element(driver, f'//*[@class="w2input_table_sb w2input_table"]//tbody//tr[{major_idx}]').click()
    wait_and_find_element(driver, '//*[@class="w2group w2tb_td btn_grp w2tb_noTH"]//*[@class="w2anchor2 btn_sch"]').click()



    scroll_op = driver.find_element(By.XPATH, '//*[@class="w2grid_scrollY"]')
    scroll_height = int(driver.find_element(By.XPATH, '//*[@class="w2grid_scrollY"]//div').size['height'])
    row_cnt = (scroll_height-268)/20

    time.sleep(0.5)
    if scroll_height == 0:
        # table_data_contents = wait_and_find_elements(driver, '//*[@id="mf_grd_main_body_tbody"]//tr')
        try:
            syllabus_list = wait_and_find_elements(driver, '//*[@id="mf_grd_main_body_tbody"]//tr//a')
        except:
            continue
        for idx, syllabus in enumerate(syllabus_list):
            syllabus.click()
            first_frame = wait_and_find_element(driver, '//iframe[@class="w2iframe "]')
            driver.switch_to.frame(first_frame)
            second_frame = wait_and_find_element(driver, '//iframe[@id="targetOzReport"]')
            driver.switch_to.frame(second_frame)

            time.sleep(1)
            wait_and_find_element(driver, '//div[@id="OZViewer"]/div/input[@title="저장"]').click()
            time.sleep(1)
            wait_and_find_element(driver, '//div[@class="ui-dialog-buttonset"]//*[2]').click()
            time.sleep(1)
            driver.switch_to.default_content()
            driver.switch_to.default_content()

            wait_and_find_element(driver,'//div[@class="w2window_header_control"]//div[@class="w2window_close"]').click()

            if idx%2 == 1:
                rename = f'./source/course/교과목개요{cnt}.xlsx'
                os.rename('./source/course/교과목 개요 출력.xlsx',rename)
                rename = f'./source/course/수업계획서{cnt}.xlsx'
                os.rename('./source/course/수업계획서 출력.xlsx',rename)
                cnt+=1
            driver.switch_to.default_content()
            
    else:
        for table_data in range(scroll_height // (row_size*20)):
            syllabus_list = wait_and_find_elements(driver, '//*[@id="mf_grd_main_body_tbody"]//tr//a')
            for idx, syllabus in enumerate(syllabus_list):
                syllabus.click()
                first_frame = wait_and_find_element(driver, '//iframe[@class="w2iframe "]')
                driver.switch_to.frame(first_frame)
                second_frame = wait_and_find_element(driver, '//iframe[@id="targetOzReport"]')
                driver.switch_to.frame(second_frame)

                time.sleep(1)
                wait_and_find_element(driver, '//div[@id="OZViewer"]/div/input[@title="저장"]').click()
                time.sleep(1)
                wait_and_find_element(driver, '//div[@class="ui-dialog-buttonset"]//*[2]').click()
                time.sleep(1)
                driver.switch_to.default_content()
                driver.switch_to.default_content()

                driver.find_element(By.XPATH,'//div[@class="w2window_header_control"]//div[@class="w2window_close"]').click()

                if idx%2 == 1:
                    rename = f'./source/course/교과목개요{cnt}.xlsx'
                    os.rename('./source/course/교과목 개요 출력.xlsx',rename)
                    rename = f'./source/course/수업계획서{cnt}.xlsx'
                    os.rename('./source/course/수업계획서 출력.xlsx',rename)
                    cnt+=1
                driver.switch_to.default_content()
            
            driver.execute_script(f"arguments[0].scrollBy(0, {row_size*20})", scroll_op) #mf_grd_main_size_y에서 스크롤 크기 확인 가능
# pd.DataFrame(class_list).to_csv('table.csv', encoding='cp949')

# for table_data in range(scroll_height // (row_size*20)):
#     time.sleep(1)
#     for row in table_data_contents:
#         row_data = row.find_elements(By.XPATH, './/td')
#         tmp_list = []
#         for shell in row_data:
#             tmp_list.append(shell.text)
#         class_list.append(tmp_list)
    
#     driver.execute_script(f"arguments[0].scrollBy(0, {row_size*20})", itemlist) #mf_grd_main_size_y에서 스크롤 크기 확인 가능

  3%|▎         | 3/86 [03:44<1:43:35, 74.88s/it]


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=129.0.6668.90)
Stacktrace:
	GetHandleVerifier [0x00397143+25587]
	(No symbol) [0x0032A2E4]
	(No symbol) [0x00221F89]
	(No symbol) [0x00267F6F]
	(No symbol) [0x0025D58D]
	(No symbol) [0x0028AB9C]
	(No symbol) [0x0025D044]
	(No symbol) [0x0028AE34]
	(No symbol) [0x002A5370]
	(No symbol) [0x0028A936]
	(No symbol) [0x0025BA73]
	(No symbol) [0x0025C4CD]
	GetHandleVerifier [0x00674C63+3030803]
	GetHandleVerifier [0x006C6B99+3366473]
	GetHandleVerifier [0x004295F2+624802]
	GetHandleVerifier [0x00430E6C+655644]
	(No symbol) [0x00332C9D]
	(No symbol) [0x0032FD68]
	(No symbol) [0x0032FF05]
	(No symbol) [0x00322336]
	BaseThreadInitThunk [0x75D1FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x779D80CE+286]
	RtlGetAppContainerNamedObjectPath [0x779D809E+238]


In [42]:
test[0].get_attribute('text')

IndexError: list index out of range